In [ ]:
from modules.searches import StrategySearcher
from datetime import datetime

searcher = StrategySearcher(
    symbol='XAUUSD',
    timeframe='H1',
    direction='buy',
    train_start=datetime(2019, 1, 1),
    train_end=datetime(2025, 1, 1),
    test_start=datetime(2022, 1, 1),
    test_end=datetime(2023, 1, 1),
    n_trials=500,
)

# Buscar estrategias usando clustering
searcher.run_search('clusters', n_models=5)

# Buscar estrategias usando causalidad
#searcher.run_search('causal', n_models=10)

# Buscar estrategias usando filtros
#searcher.run_search('filter', n_models=10)